In [1]:
import json
from pandas.io.json import json_normalize
exec(open('downloader.py').read())

In [2]:
class getInfo():
    def __init__(self, data):
        self.data = data
        
    def mergeMatchGames(self):
        storage = pd.DataFrame()
        for x in range(9):
            match = self.data[x]
            Match = pd.DataFrame({
                    'Team1': [self.getTeam1Name(match)],
                    'Team2': [self.getTeam2Name(match)],
                    'HalftimeResult': [self.getHalfResult(match)],
                    'FinalResult': [self.getEndResult(match)],
                    'GoalTimeTeam1': [self.getGoalTimeTeam1(match)],
                    'GoalTimeTeam2': [self.getGoalTimeTeam2(match)],
                    'Spectators': [self.getSpectators(match)],
                    'DateTime': [self.getDateTime(match)],
                    'Weekday': [self.getDateTime(match).dayofweek],
                    'Hour': [self.getDateTime(match).hour],
                    'PeanltyTeam1': [self.getPenaltyTeam1(match)],
                    'PenaltyTeam2': [self.getPenaltyTeam2(match)],
                    'ScorerTeam1': [self.getScorerTeam1(match)],
                    'ScorerTeam2': [self.getScorerTeam2(match)],
                    'OwnGoalTeam1': [self.getOwnGoalTeam1(match)],
                    'OwnGoalTeam2': [self.getOwnGoalTeam2(match)],
                    'OvertimeGoalTeam1': [self.getOvertimeGoalTeam1(match)],
                    'OvertimeGoalTeam2': [self.getOvertimeGoalTeam2(match)],
                })
            storage = storage.append(Match)
        return storage
   
    def getTeam1Name(self,match):
        return match['Team1']['TeamName']
    
    def getTeam2Name(self,match):
        return match['Team2']['TeamName']
    
    def getHalfResult(self,match):
        return [match['MatchResults'][0]['PointsTeam1'], match['MatchResults'][0]['PointsTeam2']]

    def getEndResult(self,match):
        return [match['MatchResults'][1]['PointsTeam1'], match['MatchResults'][1]['PointsTeam2']]
    
    def mergeInfoTeam(self, match, Info, teamNumber):
        info = np.array([])
        team = np.array([])
        for x in match['Goals']:
            team = np.append(team, x['ScoreTeam'+str(teamNumber)])
            info = np.append(info, x[Info])
        team = team[info != np.array(None)]
        info = info[info != np.array(None)]
        if(len(info)==0):
            return np.array([0])
        elif(len(team)>1):
            mask = np.append(team[0], team[1:]-team[:-1]).astype(bool)
            return info[mask]
        else:
            mask = team[0].astype(bool)
            return info[mask]
    
    def getGoalTimeTeam1(self, match):
        return self.mergeInfoTeam(match, 'MatchMinute', 1).astype(int)
    
    def getGoalTimeTeam2(self, match):
        return self.mergeInfoTeam(match, 'MatchMinute', 2).astype(int)
    
    def getPenaltyTeam1(self, match):
        return self.mergeInfoTeam(match, 'IsPenalty', 1).astype(bool)
    
    def getPenaltyTeam2(self, match):
        return self.mergeInfoTeam(match, 'IsPenalty', 2).astype(bool)
    
    def getOwnGoalTeam1(self, match):
        return self.mergeInfoTeam(match, 'IsOwnGoal', 1).astype(bool)
    
    def getOwnGoalTeam2(self, match):
        return self.mergeInfoTeam(match, 'IsOwnGoal', 2).astype(bool)
    
    def getOvertimeGoalTeam1(self, match):
        return self.mergeInfoTeam(match, 'IsOvertime', 1).astype(bool)
    
    def getOvertimeGoalTeam2(self, match):
        return self.mergeInfoTeam(match, 'IsOvertime', 2).astype(bool)
    
    def getScorerTeam1(self, match):
        return self.mergeInfoTeam(match, 'GoalGetterName', 1)
    
    def getScorerTeam2(self, match):
        return self.mergeInfoTeam(match, 'GoalGetterName', 2)
    
    def getSpectators(self,match):
        return match['NumberOfViewers']
    
    def getDateTime(self,match):
        return pd.to_datetime(match['MatchDateTime'], unit='ns', errors='ignore', box=True)

In [3]:
class converter(downloader, getInfo):
    def __init__(self):
        self.status = None
        self.lastConvMatchday = self.readLastConvMatchday()
    
    def saveLastConvMatchday(self):
        ''' Save the lastConvMatchday'''
        if(self.lastConvMatchday[1] <=1):
            lastConvMatchday = [self.lastConvMatchday[0]-1, 34]
        else:
            lastConvMatchday = [self.lastConvMatchday[0],self.lastConvMatchday[1]-1]
        with open('info.json', 'w') as file:
            file.write(json.dumps({'LastConvMatchday': lastConvMatchday}))
            
    def readLastConvMatchday(self):
        '''Read the last lastConvMatchday '''
        with open('info.json') as file:
            data = json.load(file)
        if('LastConvMatchday' in data.keys()):
            return data['LastConvMatchday']
        else:
            return downloader().beginHistory()
    
    def raiseMatchday(self):
        if(self.lastConvMatchday[1]>=34):
            self.lastConvMatchday[1] = 1
            self.lastConvMatchday[0] += 1
            print('Now loading season', self.lastConvMatchday[0])
        else:
            self.lastConvMatchday[1] += 1
            print(self.lastConvMatchday)
    
    def loadMatchday(self, year, day):
        season_day = (str(year) + '/' + str(day))
        with open(season_day+'.json', 'r') as f:#(str(year)+'/'+ str(day) + 
            return json.load(f)
    
    def convertMatchday(self, year, day):
        jsonData = self.loadMatchday(year, day)
        pandasData = getInfo(jsonData).mergeMatchGames()
        return pandasData
    
    def saveMatchday(self, data, year, day):
        file = './data/'+str(year)
        if(os.path.isfile(file)== True):
            loadData = pd.read_pickle(file)
            loadData = loadData.append(data)
            loadData.to_pickle(file)
        else:
            data.to_pickle(file)
                
    def updateSeason(self):
        lastDown = downloader().readLastLoadMatchday()
        if(lastDown > self.lastConvMatchday):
            data = self.convertMatchday(self.lastConvMatchday[0], self.lastConvMatchday[1])
            self.saveMatchday(data, self.lastConvMatchday[0], self.lastConvMatchday[1])
            self.raiseMatchday()
            self.updateSeason()
        else:
            print('Everything up to date')

In [4]:
converter().updateSeason()

[2014, 2]
[2014, 3]
[2014, 4]
[2014, 5]
[2014, 6]
[2014, 7]
[2014, 8]
[2014, 9]
[2014, 10]
[2014, 11]
[2014, 12]
[2014, 13]
[2014, 14]
[2014, 15]
[2014, 16]
[2014, 17]
[2014, 18]
[2014, 19]
[2014, 20]
[2014, 21]
[2014, 22]
[2014, 23]
[2014, 24]
[2014, 25]
[2014, 26]
[2014, 27]
[2014, 28]
[2014, 29]
[2014, 30]
[2014, 31]
[2014, 32]
[2014, 33]
[2014, 34]
Now loading season 2015
[2015, 2]
[2015, 3]
[2015, 4]
[2015, 5]
[2015, 6]
[2015, 7]
[2015, 8]
[2015, 9]
[2015, 10]
[2015, 11]
[2015, 12]
[2015, 13]
[2015, 14]
[2015, 15]
[2015, 16]
[2015, 17]
[2015, 18]
[2015, 19]
[2015, 20]
[2015, 21]
[2015, 22]
[2015, 23]
[2015, 24]
[2015, 25]
[2015, 26]
[2015, 27]
[2015, 28]
[2015, 29]
[2015, 30]
[2015, 31]
[2015, 32]
[2015, 33]
[2015, 34]
Now loading season 2016
[2016, 2]
[2016, 3]
[2016, 4]
[2016, 5]
[2016, 6]
[2016, 7]
[2016, 8]
[2016, 9]
[2016, 10]
[2016, 11]
[2016, 12]
[2016, 13]
[2016, 14]
[2016, 15]
[2016, 16]
[2016, 17]
[2016, 18]
[2016, 19]
[2016, 20]
[2016, 21]
[2016, 22]
[2016, 23]
[2016, 2